In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer
import seaborn as sns
import math
# Ignore sklearn warning for now
import warnings
warnings.filterwarnings('ignore') 
import seaborn as sns
sns.set_style("whitegrid")

In [2]:
import os
os.getcwd()

'/mnt/3106DB277DCCEAA5/Sync/Research/Coding/medicine/gods21/experiments/paris'

In [21]:
samples_summary = pd.read_excel('../../../../Data/down_syndrome/paris/Samples_summary_RNAseq.xlsx', 'Samples', index_col=1)
samples_group = samples_summary['Group']
samples_group = samples_group.dropna()
samples_group

SampleID
APOE3_D1P5_1               APOE3
APOE3_D1P6_2               APOE3
APOE3_D1P3_3               APOE3
APOE3_D1P3_4               APOE3
APOE3_D1P3_5               APOE3
                         ...    
DEAN_CTRL2_D1P3_123    DEAN_CTRL
DEAN_CTRL2_D1P4_124    DEAN_CTRL
DEAN_CTRL3_D1P3_125    DEAN_CTRL
DEAN_CTRL3_D1P3_126    DEAN_CTRL
DEAN_CTRL3_D1P4_127    DEAN_CTRL
Name: Group, Length: 106, dtype: object

In [22]:
phenotype_groups = pd.read_excel('../../../../Data/down_syndrome/paris/Samples_summary_RNAseq.xlsx', 'Group', index_col=0)
phenotype_groups

,Expected Abeta quantity in the wall of arteries,Expected Abeta quantity in the wall of capillaries,Age (year),Cell size,Expected AD,Sex,APOE
Group,,,,,,,
T21_CTRL,0.0,0.0,1.0,NaN,no,male,NaN
T21,0.0,0.0,1.0,NaN,yes,male,NaN
DEAN_DUP,3.0,3.0,64.0,NaN,yes,male,3/3
DEAN_CTRL,0.0,0.0,64.0,NaN,no,male,3/3
COR_DUP,3.0,3.0,60.0,0.000535,yes,female,3/3
COR_CTRL,0.0,0.0,60.0,"0,00045099",no,female,3/3
APP_V717I,0.5,0.0,47.0,NaN,yes,female,3/3
APP_V717I_CTRL,0.0,0.0,47.0,NaN,no,female,3/3
PSEN_R278I,NaN,NaN,60.0,NaN,yes,male,2/4


In [42]:
phenotypes = []
for sample_id, sample_group in samples_group.items():
    if sample_group in phenotype_groups.index:
        group_phenotypes = phenotype_groups.loc[sample_group]
        phenotypes.append([
            sample_id, 
            sample_group,
            group_phenotypes['Expected Abeta quantity in the wall of arteries'],
            group_phenotypes['Expected Abeta quantity in the wall of capillaries'],
            group_phenotypes['Age (year)'],
            group_phenotypes['Expected AD'],
            group_phenotypes['Sex'],
            group_phenotypes['APOE'],
        ])
phenotypes = pd.DataFrame(phenotypes, columns=['ID', 'Group', 'Abeta_arteries', 
                                               'Abeta_capillaries', 'Age',
                                               'Expected_AD', 'Sex', 'APOE'])
phenotypes.index = phenotypes['ID']
phenotypes

,ID,Group,Abeta_arteries,Abeta_capillaries,Age,Expected_AD,Sex,APOE
ID,,,,,,,,
iCTRL_WTS60_D1P2_11,iCTRL_WTS60_D1P2_11,WTS60,0.0,0.0,60.0,no,male,3/3
iCTRL_WTS60_D1P2_12,iCTRL_WTS60_D1P2_12,WTS60,0.0,0.0,60.0,no,male,3/3
iCTRL_WTS40_D1P2_13,iCTRL_WTS40_D1P2_13,WTS40,0.0,0.0,40.0,no,fermale,3/4
iCTRL_WTS40_D1P2_14,iCTRL_WTS40_D1P2_14,WTS40,0.0,0.0,40.0,no,fermale,3/4
iCTRL_C1_D1P2_15,iCTRL_C1_D1P2_15,CTRL1,0.0,0.0,NaN,no,male,3/3
...,...,...,...,...,...,...,...,...
DEAN_CTRL2_D1P3_123,DEAN_CTRL2_D1P3_123,DEAN_CTRL,0.0,0.0,64.0,no,male,3/3
DEAN_CTRL2_D1P4_124,DEAN_CTRL2_D1P4_124,DEAN_CTRL,0.0,0.0,64.0,no,male,3/3
DEAN_CTRL3_D1P3_125,DEAN_CTRL3_D1P3_125,DEAN_CTRL,0.0,0.0,64.0,no,male,3/3


In [55]:
genotype_diffexp = pd.read_csv('../../../../Data/down_syndrome/paris/all_inclusive_selection.csv', index_col=0).T
gene_exp = genotype_diffexp.loc[phenotypes.index]
gene_exp

Symbol or ID,XIST,MEG3,COL8A2,MIR4458HG,PRSS2,HLA-DRB1,TDGF1,F13A1,EREG,FOXL2NB,...,KCNV1,CRABP1,ENSG00000260159,ADRA2A,SV2C,HSD17B2,ZFP57,NPY,ZNF662,SLC30A8
ID,,,,,,,,,,,,,,,,,,,,,
iCTRL_WTS60_D1P2_11,-6.644,2.477,-6.644,-6.644,1.477,0.477,0.477,0.477,-6.644,0.477,...,5.384,3.062,3.062,2.799,2.062,3.799,6.092,-6.644,-6.644,7.192
iCTRL_WTS60_D1P2_12,-6.644,-6.644,2.676,-6.644,1.676,0.676,-6.644,-6.644,-6.644,-6.644,...,5.2,0.676,2.676,1.676,1.676,3.483,5.805,-6.644,-6.644,7.276
iCTRL_WTS40_D1P2_13,-6.644,-6.644,2.967,-6.644,-6.644,0.382,-6.644,-6.644,-6.644,1.967,...,-6.644,4.189,3.552,3.382,7.301,3.841,7.8,6.289,-6.644,0.382
iCTRL_WTS40_D1P2_14,-6.644,-6.644,2.719,-6.644,-6.644,1.982,-6.644,-6.644,-6.644,-6.644,...,0.397,5.097,1.982,1.397,6.374,4.484,8.191,6.328,1.982,-6.644
iCTRL_C1_D1P2_15,-6.644,-6.644,0.57,-6.644,-6.644,-6.644,-6.644,-6.644,-6.644,-6.644,...,-6.644,5.818,5.57,8.851,-6.644,12.462,-6.644,-6.644,6.124,-6.644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DEAN_CTRL2_D1P3_123,-6.644,-6.644,1.369,6.773,6.384,-0.216,-6.644,-6.644,4.954,-6.644,...,-6.644,-6.644,-6.644,2.784,-0.216,3.106,-6.644,-6.644,-6.644,-0.216
DEAN_CTRL2_D1P4_124,-6.644,-6.644,1.385,6.429,5.085,0.8,-6.644,-6.644,8.422,-0.2,...,-6.644,-6.644,-0.2,3.5,-6.644,4.754,-6.644,-6.644,-6.644,-6.644
DEAN_CTRL3_D1P3_125,-6.644,-6.644,1.506,6.243,1.921,-6.644,1.921,-6.644,4.506,-0.079,...,0.921,-6.644,-6.644,2.243,1.506,1.506,-6.644,-6.644,-6.644,-6.644


In [58]:
gene_description = genotype_diffexp.loc['Gene description']
gene_description

Symbol or ID
XIST         X inactive specific transcript [Source:HGNC Sy...
MEG3         maternally expressed 3 [Source:HGNC Symbol;Acc...
COL8A2       collagen type VIII alpha 2 chain [Source:HGNC ...
MIR4458HG    MIR4458 host gene [Source:HGNC Symbol;Acc:HGNC...
PRSS2        serine protease 2 [Source:HGNC Symbol;Acc:HGNC...
                                   ...                        
HSD17B2      hydroxysteroid 17-beta dehydrogenase 2 [Source...
ZFP57        ZFP57 zinc finger protein [Source:HGNC Symbol;...
NPY          neuropeptide Y [Source:HGNC Symbol;Acc:HGNC:7955]
ZNF662       zinc finger protein 662 [Source:HGNC Symbol;Ac...
SLC30A8      solute carrier family 30 member 8 [Source:HGNC...
Name: Gene description, Length: 2253, dtype: object

In [60]:
gene_exp.to_csv('../../../../Data/down_syndrome/paris/x.csv')
phenotypes.to_csv('../../../../Data/down_syndrome/paris/y.csv')
gene_description.to_csv('../../../../Data/down_syndrome/paris/genes.csv')